In [ ]:
!wget -q https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import optimizers
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, RegexpTokenizer, TweetTokenizer, PorterStemmer


from re import sub
import tokenization
from tokenization import FullTokenizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
df_train= pd.read_csv('../input/amz-data/dataset/train.csv', escapechar = "\\", quoting = 3)
df_mini= df_train[:40000]

# df_test= pd.read_csv('../input/amz-data/dataset/test.csv', escapechar = "\\", quoting = 3)

# df= pd.concat([df_train, df_test])
# df.reset_index(inplace=True)

# print(df_train.shape, df_test.shape, df.shape)

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
df_mini[:2]

In [ ]:
df_train.BROWSE_NODE_ID.unique().shape[0]

In [ ]:
df_mini.TITLE[:20000].values.shape

In [ ]:
#config
batch_size= 16
lr= 0.001
epoch= 50

In [ ]:
#tf.keras.utils.to_categorical(df_mini['BROWSE_NODE_ID'].values)
def generator(x, y, batch_size):
    samples_per_epoch = x.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    
    while True:
        #restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0
        X= x[batch_size*counter:batch_size*(counter+1)]
        Y= y[batch_size*counter:batch_size*(counter+1)].astype('float32')
        counter += 1
        yield X,Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_mini.TITLE[:25000].values,
            pd.get_dummies(df_mini.BROWSE_NODE_ID[:25000]).values, test_size=0.2, random_state=22)
print(X_train.shape, X_test.shape)
train_steps= X_train.shape[0]//batch_size
val_steps= X_train.shape[0]//batch_size

train_gen= generator(X_train, y_train, batch_size)
test_gen= generator(X_test, y_test, 8)

In [ ]:
a, b= next(test_gen)
print(a.shape, b.shape)
df_mini.BROWSE_NODE_ID[:25000].unique().shape

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(1280, activation='relu'))
model.add(tf.keras.layers.Reshape((10, 128)))
model.add(tf.keras.layers.LSTM(512, return_sequences=True))
model.add(tf.keras.layers.LSTM(512))
model.add((tf.keras.layers.Dense(df_mini.BROWSE_NODE_ID[:25000].unique().shape[0], activation='softmax')))
model.summary()

In [ ]:
name= 'AMZ_model.h5'

rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 2, verbose = 1, 
                                min_delta = 1e-4, min_lr = 1e-6, mode = 'min', cooldown=1)
        
ckp = ModelCheckpoint(name,monitor = 'val_loss',
                      verbose = 1, save_best_only = True, mode = 'min')
        
es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 8, mode = 'min', 
                    restore_best_weights = True, verbose = 1)

In [ ]:
loss= tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizers.Adam(lr=lr),loss=loss,metrics=[tf.keras.metrics.AUC(multi_label=False), 'acc'])

In [ ]:
history = model.fit_generator(train_gen, steps_per_epoch= train_steps, epochs=20)
                    #validation_data= test_gen, validation_steps=val_steps)

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot( history.history["loss"], label = "Training Loss", marker='o')
#plt.plot( history.history["val_loss"], label = "Validation Loss", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Acc")
plt.plot( history.history["acc"], label = "Training ACC" , marker='o')
#plt.plot( history.history["val_"], label = "Validation AUC", marker='+')
plt.grid(True)
plt.legend()
plt.show()